## Logging Configuration

In [1]:
from bigstream.configure_logging import configure_logging

configure_logging(None, True)

<RootLogger root (DEBUG)>

## Inputs
---

In [2]:
# reading data and working with arrays
import os
import zarr, nrrd
import numpy as np
import os

# data paths
base_path = '/Users/goinac/Work/HHMI/bioimagetools/nextflow-modules'
p = f'{base_path}/mylocal/bigstream-testdata3'
fix_path = f'{p}/fix.n5'
ref_ch = 'c0'
fix_s0_subpath = f'{ref_ch}/s0'
mov_path = f'{p}/mov.n5'
mov_s0_subpath = f'{ref_ch}/s0'
exp_factor = 1

out_p = f'{base_path}/mylocal/results/bigstream-python'
os.makedirs(out_p, exist_ok=True)

# load fix data and spacing
fix_zarr = zarr.open(store=zarr.N5Store(fix_path), mode='r')
fix_meta = fix_zarr[fix_s0_subpath].attrs.asdict()
print(f'Fix attrs: {fix_meta}')

# load mov data and spacing
mov_zarr = zarr.open(store=zarr.N5Store(mov_path), mode='r')
mov_meta = mov_zarr[mov_s0_subpath].attrs.asdict()
print(f'Mov attrs: {mov_meta}')


2024-06-18 10:26:20 - numcodecs - DEBUG - Registering codec 'zlib'
2024-06-18 10:26:20 - numcodecs - DEBUG - Registering codec 'gzip'
2024-06-18 10:26:20 - numcodecs - DEBUG - Registering codec 'bz2'
2024-06-18 10:26:20 - numcodecs - DEBUG - Registering codec 'lzma'
2024-06-18 10:26:20 - numcodecs - DEBUG - Registering codec 'blosc'
2024-06-18 10:26:20 - numcodecs - DEBUG - Registering codec 'zstd'
2024-06-18 10:26:20 - numcodecs - DEBUG - Registering codec 'lz4'
2024-06-18 10:26:20 - numcodecs - DEBUG - Registering codec 'astype'
2024-06-18 10:26:20 - numcodecs - DEBUG - Registering codec 'delta'
2024-06-18 10:26:20 - numcodecs - DEBUG - Registering codec 'quantize'
2024-06-18 10:26:20 - numcodecs - DEBUG - Registering codec 'fixedscaleoffset'
2024-06-18 10:26:20 - numcodecs - DEBUG - Registering codec 'packbits'
2024-06-18 10:26:20 - numcodecs - DEBUG - Registering codec 'categorize'
2024-06-18 10:26:20 - numcodecs - DEBUG - Registering codec 'pickle'
2024-06-18 10:26:20 - numcodecs 

In [3]:

fix_spacing_s0 = np.array(fix_meta['pixelResolution']['dimensions'][::-1]) / exp_factor
fix_spacing_s1 = fix_spacing_s0 * [2, 2, 2]
fix_spacing_s2 = fix_spacing_s0 * [4, 4, 4]
fix_spacing_s3 = fix_spacing_s0 * [8, 8, 8]
fix_spacing_s4 = fix_spacing_s0 * [16, 16, 16]
fix_spacing_s5 = fix_spacing_s0 * [32, 32, 32]
print(f'fix_spacing_s0: {fix_spacing_s0}')
print(f'fix_spacing_s1: {fix_spacing_s1}')
print(f'fix_spacing_s2: {fix_spacing_s2}')
print(f'fix_spacing_s3: {fix_spacing_s3}')
print(f'fix_spacing_s4: {fix_spacing_s4}')
print(f'fix_spacing_s5: {fix_spacing_s5}')

mov_spacing_s0 = np.array(mov_meta['pixelResolution']['dimensions'][::-1]) / exp_factor
mov_spacing_s1 = mov_spacing_s0 * [2, 2, 2]
mov_spacing_s2 = mov_spacing_s0 * [4, 4, 4]
mov_spacing_s3 = mov_spacing_s0 * [8, 8, 8]
mov_spacing_s4 = mov_spacing_s0 * [16, 16, 16]
mov_spacing_s5 = mov_spacing_s0 * [32, 32, 32]

print(f'mov_spacing_s0: {mov_spacing_s0}')
print(f'mov_spacing_s1: {mov_spacing_s1}')
print(f'mov_spacing_s2: {mov_spacing_s2}')
print(f'mov_spacing_s3: {mov_spacing_s3}')
print(f'mov_spacing_s4: {mov_spacing_s4}')
print(f'mov_spacing_s5: {mov_spacing_s5}')

global_scale = 's0'
fix_affine_spacing = fix_spacing_s0
mov_affine_spacing = mov_spacing_s0

local_scale = 's0'
fix_deform_spacing = fix_spacing_s0
mov_deform_spacing = mov_spacing_s0

fix_spacing_s0: [1. 1. 1.]
fix_spacing_s1: [2. 2. 2.]
fix_spacing_s2: [4. 4. 4.]
fix_spacing_s3: [8. 8. 8.]
fix_spacing_s4: [16. 16. 16.]
fix_spacing_s5: [32. 32. 32.]
mov_spacing_s0: [1. 1. 1.]
mov_spacing_s1: [2. 2. 2.]
mov_spacing_s2: [4. 4. 4.]
mov_spacing_s3: [8. 8. 8.]
mov_spacing_s4: [16. 16. 16.]
mov_spacing_s5: [32. 32. 32.]


## Alignment
---

### Load affine

In [4]:
# load precomputed results
affine = np.loadtxt(out_p+'/affine.mat')
affine

array([[ 9.84386100e-01, -4.44111772e-04,  1.01842376e-04,
         3.04607048e-02],
       [-1.39465551e-02,  9.96547376e-01, -6.27954979e-03,
         2.54243432e+01],
       [-5.16714859e-03, -1.13662287e-02,  9.97928648e-01,
        -4.42017886e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         1.00000000e+00]])

### Prepare cluster args

In [5]:
cluster_kwargs={
    'config':{
        'distributed.worker.memory.target':0.9,
        'distributed.worker.memory.spill':0.9,
        'distributed.worker.memory.pause':0.9,
        'distributed.comm.retry.count':10,
        'distributed.comm.timeouts.connect':'600s',
        'distributed.scheduler.worker-saturation': 1,
        'distributed.scheduler.unknown-task-duration':'60m',
    },
}

### Calculate deform transformation

In [6]:
%%time
from bigstream.piecewise_align import distributed_piecewise_alignment_pipeline

# FASTER

# get full resolution data
fix = fix_zarr[f'/{ref_ch}/{local_scale}']
mov = mov_zarr[f'/{ref_ch}/{local_scale}']

# define alignment steps
affine_kwargs = {
    'smooth_sigmas':[0.25],
    'optimizer_args':{
        'learningRate':0.25,
        'minStep':0.,
        'numberOfIterations':25,
    },
    "alignment_spacing": 1.0,
}

deform_kwargs = {
    'smooth_sigmas':[0.25],
    'control_point_spacing':50.0,
    'control_point_levels':[1],
    'optimizer_args':{
        'learningRate':2.5,
        'minStep':0.,
        'numberOfIterations':25,
    },
    "alignment_spacing": 2.0,
}

steps = [ ('affine', affine_kwargs,), ('deform', deform_kwargs,), ]

# deform
deform = distributed_piecewise_alignment_pipeline(
    fix, mov,
    fix_deform_spacing, mov_deform_spacing,
    steps=steps,
    blocksize=[128, 128, 128],
    static_transform_list=[affine,],
    write_path=out_p+'/deform.zarr',
    cluster_kwargs=cluster_kwargs,
)


2024-06-18 10:26:22 - asyncio - DEBUG - Using selector: KqueueSelector
2024-06-18 10:26:22 - distributed.http.proxy - INFO - To route to workers diagnostics web server please install jupyter-server-proxy: python -m pip install jupyter-server-proxy
2024-06-18 10:26:22 - distributed.scheduler - INFO - State start
2024-06-18 10:26:22 - distributed.scheduler - INFO -   Scheduler at: tcp://192.168.1.219:64309
2024-06-18 10:26:22 - distributed.scheduler - INFO -   dashboard at:  http://192.168.1.219:8787/status
2024-06-18 10:26:22 - distributed.scheduler - INFO - Registering Worker plugin shuffle
2024-06-18 10:26:22 - distributed.nanny - INFO -         Start Nanny at: 'tcp://192.168.1.219:64312'
2024-06-18 10:26:22 - distributed.nanny - INFO -         Start Nanny at: 'tcp://192.168.1.219:64314'
2024-06-18 10:26:22 - distributed.nanny - INFO -         Start Nanny at: 'tcp://192.168.1.219:64316'
2024-06-18 10:26:22 - distributed.nanny - INFO -         Start Nanny at: 'tcp://192.168.1.219:64318

/opt/homebrew/Caskroom/mambaforge/base/envs/bigstream/lib/python3.11/site-packages/bigstream/piecewise_align.py:384: RuntimeWarning: invalid value encountered in divide
  weights = weights / (1 - missing_weights)


2024-06-18 10:26:30 - distributed.scheduler - INFO - Receive client connection: Client-worker-c1545980-2d7e-11ef-a802-fe4548c183a7
2024-06-18 10:26:30 - distributed.core - INFO - Starting established connection to tcp://192.168.1.219:64348


/opt/homebrew/Caskroom/mambaforge/base/envs/bigstream/lib/python3.11/site-packages/bigstream/piecewise_align.py:384: RuntimeWarning: invalid value encountered in divide
  weights = weights / (1 - missing_weights)


2024-06-18 10:26:36 - distributed.scheduler - INFO - Receive client connection: Client-worker-c503397a-2d7e-11ef-a803-fe4548c183a7
2024-06-18 10:26:36 - distributed.core - INFO - Starting established connection to tcp://192.168.1.219:64350


...

2024-06-18 10:26:37 - distributed.scheduler - INFO - Receive client connection: Client-worker-c559a29c-2d7e-11ef-a805-fe4548c183a7
2024-06-18 10:26:37 - distributed.core - INFO - Starting established connection to tcp://192.168.1.219:64351


/opt/homebrew/Caskroom/mambaforge/base/envs/bigstream/lib/python3.11/site-packages/bigstream/piecewise_align.py:384: RuntimeWarning: invalid value encountered in divide
  weights = weights / (1 - missing_weights)


2024-06-18 10:26:37 - distributed.scheduler - INFO - Receive client connection: Client-worker-c58eee2a-2d7e-11ef-a804-fe4548c183a7
2024-06-18 10:26:37 - distributed.core - INFO - Starting established connection to tcp://192.168.1.219:64352


/opt/homebrew/Caskroom/mambaforge/base/envs/bigstream/lib/python3.11/site-packages/bigstream/piecewise_align.py:384: RuntimeWarning: invalid value encountered in divide
  weights = weights / (1 - missing_weights)
/opt/homebrew/Caskroom/mambaforge/base/envs/bigstream/lib/python3.11/site-packages/bigstream/piecewise_align.py:384: RuntimeWarning: invalid value encountered in divide
  weights = weights / (1 - missing_weights)


2024-06-18 10:26:38 - distributed.scheduler - INFO - Remove client Client-bd30944a-2d7e-11ef-a79b-fe4548c183a7
2024-06-18 10:26:38 - distributed.core - INFO - Received 'close-stream' from tcp://192.168.1.219:64339; closing.
2024-06-18 10:26:38 - distributed.scheduler - INFO - Remove client Client-bd30944a-2d7e-11ef-a79b-fe4548c183a7
2024-06-18 10:26:38 - distributed.scheduler - INFO - Close client connection: Client-bd30944a-2d7e-11ef-a79b-fe4548c183a7
2024-06-18 10:26:38 - distributed.scheduler - INFO - Retire worker addresses (0, 1, 2, 3, 4)
2024-06-18 10:26:38 - distributed.nanny - INFO - Closing Nanny at 'tcp://192.168.1.219:64312'. Reason: nanny-close
2024-06-18 10:26:38 - distributed.nanny - INFO - Nanny asking worker to close. Reason: nanny-close
2024-06-18 10:26:38 - distributed.nanny - INFO - Closing Nanny at 'tcp://192.168.1.219:64314'. Reason: nanny-close
2024-06-18 10:26:38 - distributed.nanny - INFO - Nanny asking worker to close. Reason: nanny-close
2024-06-18 10:26:38 - 

In [7]:
# read already saved result
deform = zarr.open(out_p+'/deform.zarr', mode='r')

In [8]:
print('Deform[0:10,0,0]',deform[0:10,0,0])

Deform[0:10,0,0] [[0.06829195 1.022542   1.5290565 ]
 [0.07427759 1.036904   1.5449455 ]
 [0.08043365 1.0498701  1.5581748 ]
 [0.08654308 1.0613896  1.568802  ]
 [0.09261805 1.0714942  1.5769047 ]
 [0.09867077 1.0802162  1.5825598 ]
 [0.10471343 1.0875878  1.5858451 ]
 [0.1107582  1.093641   1.5868376 ]
 [0.11681731 1.0984082  1.5856148 ]
 [0.12290294 1.1019213  1.5822538 ]]


In [9]:
from bigstream.piecewise_transform import distributed_apply_transform

blocksize = (64,)*3
channels = [ref_ch]

fix = fix_zarr[f'/{ref_ch}/{local_scale}']
for channel in channels:
    mov = mov_zarr[f'/{channel}/{local_scale}']
    aligned = distributed_apply_transform(
        fix, mov,
        fix_deform_spacing, mov_deform_spacing,
        transform_list=[affine, deform,],
        transform_spacing=((1,1,1), fix_deform_spacing),
        blocksize=blocksize,
        cluster_kwargs=cluster_kwargs
    )
    print(f'Write aligned result to:{out_p}/r2{channel}{local_scale}_aligned_to_r1.nrrd')
    nrrd.write(out_p+f'/r2{channel}{local_scale}_aligned_to_r1.nrrd', aligned.transpose(2,1,0), compression_level=2)


2024-06-18 10:26:38 - asyncio - DEBUG - Using selector: KqueueSelector
2024-06-18 10:26:38 - distributed.scheduler - INFO - State start
2024-06-18 10:26:38 - distributed.scheduler - INFO -   Scheduler at: tcp://192.168.1.219:64360
2024-06-18 10:26:38 - distributed.scheduler - INFO -   dashboard at:  http://192.168.1.219:8787/status
2024-06-18 10:26:38 - distributed.scheduler - INFO - Registering Worker plugin shuffle
2024-06-18 10:26:38 - distributed.nanny - INFO -         Start Nanny at: 'tcp://192.168.1.219:64363'
2024-06-18 10:26:38 - distributed.nanny - INFO -         Start Nanny at: 'tcp://192.168.1.219:64367'
2024-06-18 10:26:38 - distributed.nanny - INFO -         Start Nanny at: 'tcp://192.168.1.219:64365'
2024-06-18 10:26:38 - distributed.nanny - INFO -         Start Nanny at: 'tcp://192.168.1.219:64369'
2024-06-18 10:26:38 - distributed.nanny - INFO -         Start Nanny at: 'tcp://192.168.1.219:64371'
2024-06-18 10:26:39 - distributed.scheduler - INFO - Register worker <Work